In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)});
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['adult.csv']


In [12]:
df = pd.read_csv('../input/adult.csv')
df.head()
#классификация 

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [13]:
X = df.drop('income', axis=1)
y = df['income'].map({'<=50K':0, '>50K':1})

In [14]:
# Создаём список количественных признаков
num_features = ['age', 'fnlwgt', 'educational-num', 
                'capital-gain', 'capital-loss', 'hours-per-week']
# X_num -- датафрейм только с количественными признаками
X_num = X[num_features]
# X_cat -- датафрейм только с категориальными признаками
X_cat = X.drop(num_features, axis=1)
# Проверим, ничего ли не потеряли
print(X.shape, X_num.shape, X_cat.shape)

(48842, 14) (48842, 6) (48842, 8)


In [15]:
# Меточное кодирование
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
X_cat_new = oe.fit_transform(X_cat)

In [16]:
# В результате -- числовые признаки, (но бессмысленные)
X_cat_new

array([[ 4.,  1.,  4., ...,  2.,  1., 39.],
       [ 4., 11.,  2., ...,  4.,  1., 39.],
       [ 2.,  7.,  2., ...,  4.,  1., 39.],
       ...,
       [ 4., 11.,  6., ...,  4.,  0., 39.],
       [ 4., 11.,  4., ...,  4.,  1., 39.],
       [ 5., 11.,  2., ...,  4.,  0., 39.]])

In [17]:
# Теперь склеим нашу таблицу обратно: hstack = horizontal stack, 
# т. е. поставить две матрицы рядом по горизонтали
X_new = np.hstack([X_num.values, X_cat_new])
# Размерность совпадает с исходной
X_new.shape

(48842, 14)

In [19]:
# Продолжаем далее...
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X_new)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_test))

0.8289247399885349


In [21]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn = KNeighborsClassifier(n_neighbors=7)
scores = cross_val_score(knn, X_train, y_train, 
                         cv=kf, scoring='accuracy')
print(scores)
mean_score = scores.mean()
print(mean_score)

[0.83185478 0.83401583 0.83073983 0.83346983 0.83128583]
0.8322732226376284


In [23]:
# from sklearn.model_selection import GridSearchCV
# knn_params = {'n_neighbors': np.arange(1, 21)} # число соседей -- от 1 до 20
# knn_grid = GridSearchCV(knn, 
#                         knn_params, 
#                         scoring='accuracy',
#                         cv=5) # или cv=kf
# knn_grid.fit(X_train, y_train)

In [24]:
for n in range(1, 20, 5):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    knn = KNeighborsClassifier(n_neighbors=n)
    scores = cross_val_score(knn, X_train, y_train, cv=kf, scoring='accuracy')
    mean_score = scores.mean()
    print(n, mean_score)

1 0.7970298300583548
6 0.8334471946045579
11 0.8341568947137393
16 0.8370232783498786


In [25]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 20, 5)} # число соседей -- от 1 до 20
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=5) # или cv=kf
knn_grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=16, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 1,  6, 11, 16])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)